In [1]:
import os
import sys
import re
import string

import random
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
import emoji
import contractions

from sklearn import datasets
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stops.discard('not')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from cardinal.random import RandomSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from cardinal.uncertainty import ConfidenceSampler
from cardinal.utils import ActiveLearningSplitter
from cardinal.clustering import KMeansSampler
from cardinal.batch import RankedBatchSampler

import tensorflow_text as text  # Registers the ops.
import tensorflow_hub as hub

import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

2024-08-22 13:28:25.080637: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 13:28:25.081634: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-22 13:28:25.084296: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-22 13:28:25.091899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 13:28:25.104069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

Codes in considerations:
- https://github.com/ritun16/Machine_Learning_short_projecct/blob/master/Active_Learning/Active_Learning_Data_Annotation.ipynb
- https://github.com/zhao-shuyang/active_learning
- https://dataiku-research.github.io/cardinal/auto_examples/plot_random_vs_confidence.html
- https://dataiku-research.github.io/cardinal/auto_examples/plot_hard_to_classify.html

In [206]:
tf.__version__

'2.16.1'

In [207]:
digits = datasets.load_digits()

In [208]:
digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [241]:
digits['data']

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

## The Data

In [4]:
full_data = pd.read_csv('../data/orig/original_full_dataset.csv')
full_data = full_data[['uniqueID', 'text']]
full_data['label'] = ''
full_data

,uniqueID,text,label
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,
...,...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis...",
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...,
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t...",


In [5]:
#pilot = pd.read_csv('data/pilot/Annotation_Chico 1.csv', encoding='unicode_escape')

#temp = pd.read_csv('data/pilot/Annotations_Aditi 1.csv')
#pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

pilot = pd.read_csv('../data/pilot/Annotations_Aditi 1.csv')

temp = pd.read_csv('../data/pilot/Annotations_Aditi 2.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

temp = pd.read_csv('../data/pilot/Annotations_Aditi 3.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

temp = pd.read_csv('../data/pilot/Annotations_Aditi 4.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

temp = pd.read_csv('../data/pilot/Annotations_Susan 1.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

temp = pd.read_csv('../data/pilot/Annotations_Susan 2.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

temp = pd.read_csv('../data/pilot/Annotations_Susan 3.csv')
pilot = pd.concat([pilot[['text', 'label']], temp[['text', 'label']]], axis = 0)

pilot['uniqueID'] = pilot.index
pilot['uniqueID'] = 'antisexm24_uk_remaining-' + pilot['uniqueID'].astype(str)
pilot.insert(0, 'uniqueID', pilot.pop('uniqueID'))  ## repositioning it to the front

pilot

,uniqueID,text,label
0,antisexm24_uk_remaining-0,She has NOTHING on Braverman. Nothing. \n\nSue...,sexism
1,antisexm24_uk_remaining-1,@10DowningStreet @SuellaBraverman @ukhomeoffic...,neither
2,antisexm24_uk_remaining-2,@_TheGMan @benandjerrysUK @SuellaBraverman Goo...,neither
3,antisexm24_uk_remaining-3,@Jerry_grey2002 @drmikechao @TsarKastik But I ...,neither
4,antisexm24_uk_remaining-4,@10DowningStreet @SuellaBraverman @ukhomeoffic...,neither
...,...,...,...
95,antisexm24_uk_remaining-95,Boris Johnson visited Gigg Lane stadium today....,neither
96,antisexm24_uk_remaining-96,@EmilyThornberry @PETAUK This at labour won’t ...,neither
97,antisexm24_uk_remaining-97,Queen 👑 @AngelaRayner https://t.co/7HDiEHy3sJ,neither
98,antisexm24_uk_remaining-98,UK Prime Minister Boris Johnson lashed out at ...,neither


In [10]:
data = pd.concat([pilot, full_data], axis=0)
data = data.drop_duplicates(subset='text')
data['label'] = data[['label']].replace(r'^\s*$', np.nan, regex=True)
data

,uniqueID,text,label
0,antisexm24_uk_remaining-0,She has NOTHING on Braverman. Nothing. \n\nSue...,sexism
1,antisexm24_uk_remaining-1,@10DowningStreet @SuellaBraverman @ukhomeoffic...,neither
2,antisexm24_uk_remaining-2,@_TheGMan @benandjerrysUK @SuellaBraverman Goo...,neither
3,antisexm24_uk_remaining-3,@Jerry_grey2002 @drmikechao @TsarKastik But I ...,neither
4,antisexm24_uk_remaining-4,@10DowningStreet @SuellaBraverman @ukhomeoffic...,neither
...,...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis...",NaN
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,NaN
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...,NaN
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t...",NaN


In [9]:
labeled_data = data[data['label'].notna()]
unlabeled_data = data[data['label'].isna()]

unlabeled_data

,uniqueID,text,label
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,NaN
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",NaN
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,NaN
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,NaN
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,NaN
...,...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis...",NaN
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,NaN
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...,NaN
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t...",NaN


In [213]:
X_labelled = labeled_data['text']
y_labelled = labeled_data['label']

X_unlabelled = unlabeled_data['text']
X_unlabelled = X_unlabelled[238:] # 1st batch
X_unlabelled

238       if the Daily Mail thinks its outrageous that K...
239       BREAKING: As foreign-born criminal Boris Johns...
240       As I said recently, Suella Braverman is absolu...
241       Liz Truss might want to give it a few days bef...
242       I admit I have still not come to terms with th...
                                ...                        
115186    @MichelleDewbs @trussliz She was clear, concis...
115187    @SamCoatesSky @RupaHuq @SuellaBraverman Extrao...
115188    Every time Liz Truss talks about her mandate o...
115189    @itaisher I think I just mean the "de facto" t...
115190    @HappyUlster @jameshhowe @13MoonWillow @benhab...
Name: text, Length: 114942, dtype: object

In [214]:
def usual_preprocess(text):

    sent = []
    words = text.split()
    for word in words:
        word_result = word
        if word.startswith("http"):
            word_result = '[URL]'
        else:
            pass
        sent.append(word_result)
    text = " ".join(sent)

    ## replace some general text alterations which would otherwise go undetected
    text = text.replace(r"&amp;", "and")

    text = text.replace(r"\n", "")
    

    text = emoji.demojize(text)

    ## expanding on contractions
    text = contractions.fix(text)

    ## remove email IDs if present
    text = re.sub("[\w]+@[\w]+\.[c][o][m]", "[EMAIL]", text)

    ## replacing usernames in the text with generic keyword '@USERNAME'
    #text=remove_username_mentions(text)


    # removing unicode emojis from the text
    # at the end for removing any trailing emoji patterns which add no value to the text
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # remove extra spacing from text
    text = re.sub(' +', ' ', text)

    return text

In [215]:
chars = re.escape(string.punctuation)

X_labelled = X_labelled.apply(lambda x: re.sub('['+chars+']', '',x))
X_labelled = X_labelled.apply(lambda x: re.sub('[!@#$%^&*()[]{};:,./<>?\|`~-=_+]', '',x))
# since it did not undergo preprocessing
X_labelled = X_labelled.apply(lambda x: usual_preprocess(x))

X_unlabelled = X_unlabelled.apply(lambda x: re.sub('['+chars+']', '',x))
X_unlabelled = X_unlabelled.apply(lambda x: re.sub('[!@#$%^&*()[]{};:,./<>?\|`~-=_+]', '',x))

In [231]:
#le = preprocessing.LabelEncoder()
#model = RandomForestClassifier()
model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('logit', LogisticRegression(solver='lbfgs', n_jobs=-1))
])
batch_size = 1000
sampler = ConfidenceSampler(model, batch_size)

In [232]:
#X_labelled = le.fit_transform(X_labelled)
#X_unlabelled = le.fit_transform(X_unlabelled)

In [233]:
X_labelled

0     She has NOTHING on Braverman Nothing Suella Br...
1     10DowningStreet SuellaBraverman ukhomeoffice W...
2     TheGMan benandjerrysUK SuellaBraverman Good on...
3     Jerrygrey2002 drmikechao TsarKastik But I have...
4     10DowningStreet SuellaBraverman ukhomeoffice O...
                            ...                        
95    Boris Johnson visited Gigg Lane stadium today ...
96    EmilyThornberry PETAUK This at labour will not...
97                     Queen :crown: AngelaRayner [URL]
98    UK Prime Minister Boris Johnson lashed out at ...
99    ladyhaja AngelaRayner Most debating I ever saw...
Name: text, Length: 800, dtype: object

In [234]:
X_train, X_test, y_train, y_test =  train_test_split(X_labelled, y_labelled, test_size=0.2, random_state=123)
model.fit(X_train, y_train)  
sampler.fit(X_train, y_train)
selected = sampler.select_samples(X_unlabelled)

In [235]:
accuracies = []

# Evaluating performance
accuracies.append(model.score(X_test, y_test))

In [236]:
accuracies

[0.89375]

In [222]:
selected

array([ 25173, 101800,   5837,  95577,  59925, 105604,  67677, 112937,
        29456,   5663,  22593,  10645,  81087,  93567, 108467,  85316,
        88695,  50284,  31760,  84580,  90951,   7201,  78613,  47548,
        71565,  13296, 104244,  62675,  69997,  82125, 112736,  61296,
        43257,  43613, 113940, 110423,  57301,  49457,  14645,  46241,
         1449,  99014,  40718,  54199,  15388,  11668,  16811,  17315,
        27196,  93662,  40944,  95815,  73984,   7820, 106798, 113485,
        33141,  99258,  95517,  29602,  72827,  64147,  34355,   8541,
        17280, 106643,  33499,   1618, 109684,  54184,  68281,   1023,
        96508,  33262,  80923,  42634,  95299,  73767,  87208,   9235,
        69238,  67986,  94967, 114678,  62336,  45968,  75881,  77180,
        47145,  33102,  28616,  58547,  58151,  39632,   1223,  79210,
        86840,  85897,  80249,  47304,  27312,  75449,  60795,  13952,
        50495,  94608,  78563,  53498,  37224, 113372, 113476,  34709,
      

In [ ]:
# Updating the labeled and unlabeled pool
'''X_labelled = np.concatenate([X_labelled, selected])
# The selected samples are sent to be labeled as y_selected
y_labelled = np.concatenate([y_labelled, y_selected])'''

Will not use the above method as it gives random IDs

### Measuring hard to classify examples

In [ ]:
def get_initial_labelled_dataset(num_samples_per_class=5):
  """
  Getting the initial balanced dataset. Although this is random in this code.
  In actual production system it should be informative labelled dataset given by the annotators/SMEs.

  Args:
    num_samples_per_class: int, number of samples to be takes per class in the first iteration.

  Returns:
    numpy array of labelled dataset with true labels and pooled dataset with true labels (for performance checking)
  """

  X = list()
  y = list()
  X_pooled = list()
  y_pooled = list()
  labelled_idx = list()

  counter_dict = dict()

  for idx, target in enumerate(digits['target']):
    if target in counter_dict:
      if counter_dict[target] == num_samples_per_class:
        continue
      counter_dict[target] += 1
    else:
      counter_dict[target] = 1
    X.append(digits['data'][idx])
    y.append(target)
    labelled_idx.append(idx)

  X_pooled = np.delete(digits['data'], labelled_idx, axis=0)
  y_pooled = np.delete(digits['target'], labelled_idx)

  return np.asarray(X), np.asarray(y), X_pooled, y_pooled